# Classification of Images between Dogs and Cats

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
from shutil import copyfile

In [ ]:
print (len(os.listdir('Data/PetImages/Cat/')))
print (len(os.listdir('Data/PetImages/Dog/')))

# Creating the directory structure for the Training and Test Data

In [ ]:
os.mkdir('Data/cats-v-dogs')
os.mkdir('Data/cats-v-dogs/training')
os.mkdir('Data/cats-v-dogs/testing')
os.mkdir('Data/cats-v-dogs/training/dogs')
os.mkdir('Data/cats-v-dogs/training/cats')
os.mkdir('Data/cats-v-dogs/testing/dogs')
os.mkdir('Data/cats-v-dogs/testing/cats')

In [ ]:
# Function to split the data into training and test sets with randomized shuffle

def split_data(Source,Training,Testing,Split_size):
    files = []
    for filename in os.listdir(Source):
        file = Source + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print (filename + " " + "is zero size file")
    # Splitting based on random set
    training_length = int(len(files)*Split_size)
    testing_length = int(len(files)-training_length)
    shuffled_set = random.sample(files,len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]
    
    #copying the file 
    for filename in training_set:
        this_file = Source + filename
        destination = Training + filename
        copyfile(this_file,destination)
    for filename in testing_set:
        this_file = Source + filename
        destination = Testing + filename
        copyfile(this_file,destination)
        
Cat_Source_Dir = 'Data/PetImages/Cat/'
Cat_Training_Dir = 'Data/cats-v-dogs/training/cats/'
Cat_Testing_Dir = 'Data/cats-v-dogs/testing/cats/'
Dog_Source_Dir = 'Data/PetImages/Dog/'
Dog_Training_Dir = 'Data/cats-v-dogs/training/dogs/'
Dog_Testing_Dir = 'Data/cats-v-dogs/testing/dogs/'

split_size = 0.9
split_data(Cat_Source_Dir,Cat_Training_Dir,Cat_Testing_Dir,split_size)
split_data(Dog_Source_Dir,Dog_Training_Dir,Dog_Testing_Dir,split_size)


     

In [ ]:
print (len(os.listdir('Data/cats-v-dogs/training/dogs/')))
print (len(os.listdir('Data/cats-v-dogs/training/cats/')))
print (len(os.listdir('Data/cats-v-dogs/testing/dogs/')))
print (len(os.listdir('Data/cats-v-dogs/testing/cats/')))

# Building a simple CNN model

In [ ]:
model = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
model.compile(optimizer=RMSprop(lr=0.001),loss='binary_crossentropy',metrics=['acc'])

# Data through Image Generator

In [ ]:
Training_Dir = 'Data/cats-v-dogs/training'
train_gen = ImageDataGenerator(rescale=1.0/255.)
training_generator = train_gen.flow_from_directory(directory=Training_Dir,batch_size=100,class_mode='binary',target_size=(150,150))



Validation_Dir ='Data/cats-v-dogs/testing/'
validation_gen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_gen.flow_from_directory(directory=Validation_Dir,batch_size=100,class_mode='binary',target_size=(150,150))

In [ ]:
history = model.fit_generator(training_generator,epochs=20,verbose=1,validation_data=validation_generator)